In [1]:
import rag_functions as rf

In [2]:
import os
import chromadb
from chromadb.utils import embedding_functions

In [3]:
from together import Together
from openai import OpenAI

In [4]:
DATA_PATH = "/Users/sladkydrevo/opt/baka/dataset/texts"
QUESTIONS_PATH = "/Users/sladkydrevo/opt/baka/dataset/questions"

In [ ]:
GENERATED_ANSWERS_FOLDER_PATH = "/Users/sladkydrevo/opt/baka/generated_answers_v2"

In [ ]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
TOGETHER_API_KEY = os.environ["TOGETHER_API_KEY"]

In [7]:
texts = rf.load_texts(DATA_PATH)
chunk_counts, chunk_data = rf.chunk_texts(texts, chunk_size=128, overlap=10)
chunk_names, text_chunks = rf.split_dict_data(chunk_data) 

In [8]:
questions_data = rf.load_texts(QUESTIONS_PATH)
questions = rf.convert_questions_dict(questions_data)
question_names, question_texts = rf.split_dict_data(questions) 

In [9]:
class Chroma:
    def __init__(self, collection_name, documents, ids):
        self.collection_name = collection_name
        self.documents = documents
        self.ids = ids
        self.chroma_client = chromadb.Client()
        
    def _get_embedding_function(self, model_name):
        if model_name == "paraphrase-multilingual-mpnet-base-v2":
            ef = embedding_functions.SentenceTransformerEmbeddingFunction(
                model_name=model_name
            )
            
        elif model_name == "text-embedding-3-large":
            ef = embedding_functions.OpenAIEmbeddingFunction(
                model_name=model_name,
                api_key=OPENAI_API_KEY,
            )
        return ef
        
    def populate_db(self, embedding_model):
        ef = self._get_embedding_function(embedding_model)
        try:
            self.chroma_client.delete_collection(self.collection_name)
        except ValueError:
            pass
            
        self.collection = self.chroma_client.create_collection(name=self.collection_name, embedding_function=ef)
        self.collection.upsert(
            documents=self.documents,
            ids=self.ids
        )
        print(f"Documents embedded and inserted into collection.")
        
    def get_results(self, question, n_results):
        outputs = self.collection.query(
            query_texts=[question], 
            n_results=n_results
        )
        self.outputs = outputs
        return outputs

In [10]:
def inject_prompt(context, question):
    prompt = f"Pouze na základě následujícího kontextu odpověz na otázku. Odpověď by neměla být delší než jeden krátký odstavec. Pokud jsi nenašel odpověď, napiš 'Nevím.'\n\nKontext:\n{context}\n\nOtázka: {question}"
    return prompt

In [ ]:
class RAGtest(Chroma):
    def __init__(self, collection_name, documents, ids):
        super().__init__(collection_name, documents, ids)

    def generate_answer(self, model_name, prompt):
        if model_name == "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free":
            client = Together(api_key=TOGETHER_API_KEY)
            
        elif model_name == "gpt-4o-mini":
            client = OpenAI(api_key=OPENAI_API_KEY)
            
        else:
            raise Exception("Wrong model name.")
        
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
        )
        self.answer = response.choices[0].message.content
        return self.answer
    
    def generate_all_answers(self, llm_family, questions, n_results):
        answers = {}
        for question in questions:
            output_texts = self.get_results(
                question=question, 
                n_results=n_results
            )
            context = "\n\n".join(output_texts["documents"][0])
            prompt = inject_prompt(context, question)
            answer = self.generate_answer(llm_family, prompt)
            print(f"Question: {question}\nAnswer:\n{answer}")
            answers[question] = answer
        return answers

In [12]:
rag = RAGtest(
    collection_name="dataset",
    documents=text_chunks, 
    ids=chunk_names
)

In [ ]:

embedding_models = {
    "paraphrase" : "paraphrase-multilingual-mpnet-base-v2", 
    "openai" : "text-embedding-3-large"
}

llms = {
    "llama" : "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free", 
    "gpt" : "gpt-4o-mini"
}

for ef_family, ef in embedding_models.items():
    rag.populate_db(embedding_model=ef)
    
    for llm_family, llm in llms.items():
        print(f"Testing combination {ef_family} and {llm_family}...")
        answers = rag.generate_all_answers(
            llm_family=llm, 
            questions=question_texts,
            n_results=3
        )        
        filename = f"{ef_family} + {llm_family}.json"
        path = os.path.join(GENERATED_ANSWERS_FOLDER_PATH, filename)
        rf.save_json(answers, path)
        print("Generated answers saved to file.")


Documents embedded and inserted into collection.
Testing combination paraphrase and llama...
Question: Jaký byl výsledek měření teploty na povrchu objektu na okraji Sluneční soustavy a co to naznačuje?
Answer:
Měření teploty na povrchu objektu X7-N1 ukázalo nepřirozené výkyvy teploty, kdy teplota stoupla o několik stupňů během několika hodin, což je neobvyklé pro objekt ve vzdálenosti 400 milionů kilometrů od Slunce. To naznačuje, že objekt může obsahovat aktivní geologické procesy nebo neznámý zdroj energie.
Question: Existují nějaké známé přírodní jevy, které by tajemný signál u Proximy Centauri mohly způsobit?
Answer:
Ano, existují přírodní jevy, které by mohly způsobit tajemný signál u Proximy Centauri. Mezi ně patří atmosférické procesy nebo interakce s hvězdným větrem vycházejícím z Proximy Centauri.
Question: Jaké chemické sloučeniny byly na povrchu jednoho z měsíců Sluneční soustavy detekovány?
Answer:
Nevím.
Question: Jakým teleskopem budou vědci zkoumat podrobněji světelné im